<a href="https://colab.research.google.com/github/arindam15964/mlrepo1/blob/master/mutipleclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing modules
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
import cv2
import numpy as np
import numpy as np
from keras.preprocessing import image
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import tensorflow as tf
IMG_SIZE=256

Using TensorFlow backend.


In [0]:
#creating train data
import random
id_array=["0","1","2","3","4"]
#ls=[0,0,0,0,0]
train_data=[]
i=0
j=0
k=0
l=0
m=0
path2="/content/train"
def create_train_data():
  i=0
  j=0
  k=0
  l=0
  m=0
  for category in id_array:
    newpath=os.path.join(path2,category)
    class_num=id_array.index(category)
    for img in os.listdir(newpath):
        img_array=cv2.imread(os.path.join(newpath,img),cv2.IMREAD_COLOR )
        new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        #plt.imshow(new_array,cmap="gray")
        #plt.show()
        #break
        if class_num==0 :
          i=i+1
          if i<=144:
            train_data.append([new_array,class_num])
        elif class_num==1 :
          j=j+1
          if j<=144:
           train_data.append([new_array,class_num])
        elif class_num==2 :
          k=k+1
          if k<=144:
           train_data.append([new_array,class_num])
        elif class_num==3 :
          l=l+1
          if l<=144:
           train_data.append([new_array,class_num])
        elif class_num==4 :
          m=m+1
          if m<=144:
           train_data.append([new_array,class_num])
        
create_train_data()
random.shuffle(train_data)

In [4]:
print(len(train_data))

720


In [5]:
print(i)
print(j)
print(k)
print(l)
print(m)

0
0
0
0
0


In [6]:
X=[]
y=[]
i=0  
for features , labels in train_data:
  X.append(features)
  y.append(labels)
depth = 5
print(len(X))
one_hot_y=tf.one_hot(y, depth)
ohy=one_hot_y.numpy()
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X=X/255.0

720


In [7]:
print(ohy)

[[0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, ohy, test_size=0.1, random_state=42)

In [0]:
#creating the model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE,IMG_SIZE, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3),padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Conv2D(32, (3, 3),padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu'))
# model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 5, activation = 'softmax'))

In [0]:
opt = keras.optimizers.Adagrad(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=32),
                        epochs=20,
                        validation_data=(X_test, y_test),
                        workers=4)

Epoch 1/20
21/21 [==============================] - 62s 3s/step - loss: 2.4602 - accuracy: 0.2068 - val_loss: 1.6078 - val_accuracy: 0.2778
Epoch 2/20
21/21 [==============================] - 57s 3s/step - loss: 1.6005 - accuracy: 0.2083 - val_loss: 1.5996 - val_accuracy: 0.2917
Epoch 3/20
21/21 [==============================] - 58s 3s/step - loss: 1.5676 - accuracy: 0.2948 - val_loss: 1.5481 - val_accuracy: 0.2639
Epoch 4/20
21/21 [==============================] - 58s 3s/step - loss: 1.5376 - accuracy: 0.3410 - val_loss: 1.5135 - val_accuracy: 0.3750
Epoch 5/20
21/21 [==============================] - 57s 3s/step - loss: 1.5155 - accuracy: 0.3256 - val_loss: 1.5214 - val_accuracy: 0.2917
Epoch 6/20
21/21 [==============================] - 59s 3s/step - loss: 1.4919 - accuracy: 0.3333 - val_loss: 1.5003 - val_accuracy: 0.3889
Epoch 7/20
21/21 [==============================] - 57s 3s/step - loss: 1.4783 - accuracy: 0.3488 - val_loss: 1.5025 - val_accuracy: 0.4028
Epoch 8/20
21/21 [==

In [13]:

# Predicting USing the model
sdf=pd.read_csv("/content/sample.csv")
img_array=sdf['image']
pathf="/content/test/"
y_f=[]
for image1 in img_array:
  newpath=pathf+image1
  img=image.load_img(newpath,target_size=(IMG_SIZE,IMG_SIZE))
  img=image.img_to_array(img)
  img=np.expand_dims(img,axis=0)
  img=img/255
  value=model.predict(img)
  y_f.append(value)
  z_f=[]
for a in y_f:
  for data in a:
    z=np.argmax(data, axis=None)
    z_f.append(z)
    
  
dict={
    'image': img_array,
    'label': z_f
}
dff=pd.DataFrame.from_dict(dict)
dff.to_csv("ans.csv",index=False)
print(dff)

       image  label
0      0.jpg      1
1      1.jpg      1
2     10.jpg      1
3     11.jpg      2
4     12.jpg      1
..       ...    ...
276  276.jpg      1
277  277.jpg      2
278  278.jpg      2
279  279.jpg      1
280  280.jpg      2

[281 rows x 2 columns]
